<a href="https://colab.research.google.com/github/genaiconference/RAG_Workshop_DHS2024/blob/main/10_RAG_with_open_source_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced Retrieval-Augmented Generation (RAG): LangChain Implementation
This Notebook implements an advanced RAG system with the following components
* How to handle **multiple data sources**
* Showcases the power of various **Advanced Retrievers**

In [8]:
!git clone https://github.com/genaiconference/RAG_Workshop_DHS2024.git

Cloning into 'RAG_Workshop_DHS2024'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (88/88), done.
remote: Total 90 (delta 34), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (90/90), 29.54 MiB | 9.09 MiB/s, done.
Resolving deltas: 100% (34/34), done.


In [1]:
!pip install -q langchain==0.0.309 chromadb==0.4.15 pypdf==3.16.2 openai==0.28.1 tiktoken==0.5.1 rank_bm25==0.2.2 sentence_transformers==2.2.2 langchain-community sentencepiece transformers accelerate einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.8/479.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

Define open source LLM

In [2]:
import torch
torch.cuda.is_available()

import torch
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

repo_id = 'microsoft/Phi-3-mini-4k-instruct'

model_path = snapshot_download(repo_id=repo_id,repo_type="model", local_dir="./phi-3", local_dir_use_symlinks=False)

model_path = "./phi-3"

tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

# Needs 12GB of vRAM to run in float32 (default)
# Run this line to load in float16. You need Gb of vRAM
torch.set_default_dtype(torch.float16)

model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1194: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/1.10k [00:00<?, ?B/s]

SECURITY.md:   0%|          | 0.00/2.70k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# from langchain_huggingface.llms import HuggingFacePipeline

from langchain import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=10)
llm = HuggingFacePipeline(pipeline=pipe)

In [5]:
llm.invoke("what is 2+2?")

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.


'\n\n# Answer\n2+2 equals '

Define open source Embedding function

In [6]:
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings


embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

.gitattributes:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [7]:
# @title
# wrap colab output text

from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

## Step1: Document Loading
Load Leave Policy Document

In [9]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path='/content/RAG_Workshop_DHS2024/data/misc/Leave-Policy-India.pdf')
documents = loader.load()
len(documents)

8

## Step2: Document Splitting

In [10]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

def chunkByWord(text):
    return len(text.split(" "))

text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, length_function = chunkByWord, chunk_overlap=50, add_start_index = False)

docs = text_splitter.split_documents(documents)

print(len(docs))

8


## Step3: Vector Store Creation

In [11]:
## save to disk
from langchain.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings, persist_directory='vectorstore/open_source_leave_policy/')

## load from disk
# vectorstore = Chroma(persist_directory='vectorstore/open_source_leave_policy/',embedding_function=embeddings)

## Step4: Retrieval

### Type1 - LOTR (Merger Retriever) - Handling multiple retrievers
Lord of the Retrievers, also known as MergerRetriever, takes a list of retrievers as input and merges the results of their get_relevant_documents() methods into a single list. The merged results will be a list of documents that are relevant to the query and that have been ranked by the different retrievers.

The MergerRetriever class can be used to improve the accuracy of document retrieval in a number of ways. First, it can combine the results of multiple retrievers, which can help to reduce the risk of bias in the results. Second, it can rank the results of the different retrievers, which can help to ensure that the most relevant documents are returned first.

In [12]:
from langchain.retrievers import MergerRetriever, BM25Retriever

# Define 2 diff retrievers with 2 diff search type.
retriever_1 = vectorstore.as_retriever(search_type="similarity",
                                                 search_kwargs={"k": 5, "include_metadata": True}
                                                )

# retriever_2 = vectorstore.as_retriever(search_type="mmr",
#                                                  search_kwargs={"k": 5, "include_metadata": True}
#                                                 )

# BM25 Retriver for keyword based retrieval using BM25 algorithm
bm25_retriever = BM25Retriever.from_documents(docs)

bm25_retriever.k = 3 # set the num of docs to be retrieved

# Hybrid Retriver using Semantic Search and Keyword search together
lotr = MergerRetriever(retrievers=[retriever_1, bm25_retriever])

In [14]:
from RAG_Workshop_DHS2024.utils import *

In [19]:
!pip install --upgrade langchain==0.0.343

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.2/188.2 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.1.23
    Uninstalling langchain-core-0.1.23:
      Successfully uninstalled langchain-core-0.1.23
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.309
    Uninstalling langchain-0.0.309:
      Successfully uninstalled langchain-0.0.309
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-community 0.0.20 requires langchain-core<0.2,>=0.1.21, but you have langchain-core 0.0.13 which is incompatible.


In [22]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="/content/.env")

True

In [30]:
from langchain.chains import ConversationalRetrievalChain
from IPython.display import Markdown

chain = ConversationalRetrievalChain.from_llm(llm=llm,
                                              combine_docs_chain_kwargs={"prompt": RAG_prompt},
                                              retriever=lotr,
                                              memory=memory,
                                              return_source_documents=True)

In [ ]:
query = "Compare the Revenue trends of Microsoft between the years 2022 and 2023?"
result = chain({"question": query})
result